### Setup

In [1]:
import sqlite3
import pandas as pd
import numpy as np
# import pysradb as psra
from pysradb.search import SraSearch, EnaSearch, GeoSearch

/home/jho5ze/.local/lib/python3.7/site-packages/pysradb/utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Pysradb approach

In [2]:
x = SraSearch(return_max=150, query="MG1655 RNA-Seq paired stranded") #Escherichia dRNA-Seq")
x.search()
x = x.get_df()
# x.sample_scientific_name = x.sample_scientific_name.apply(lambda row: row.replace(" str. ", " strain="))#.run_1_accession.unique()
x.study_accession.unique()

100%|██████████| 1/1 [00:00<00:00,  5.33it/s]


array(['SRP235283'], dtype=object)

In [3]:
x

,study_accession,experiment_accession,experiment_title,sample_taxon_id,sample_scientific_name,experiment_library_strategy,experiment_library_source,experiment_library_selection,sample_accession,sample_alias,experiment_instrument_model,pool_member_spots,run_1_size,run_1_accession,run_1_total_spots,run_1_total_bases
0,SRP235283,SRX7291196,GSM4211192: WT rep2_21°C; Escherichia coli; RN...,562,Escherichia coli,RNA-Seq,TRANSCRIPTOMIC,cDNA,SRS5784357,GSM4211192,Illumina HiSeq 2000,17461337,790322623,SRR10612035,17461337,1729601877


In [7]:
x[x.study_accession == "SRP235283"].to_csv("Escherichia_coli_representative_sample_info.csv")

In [8]:
x[(x.run_1_accession.apply(lambda row: row.replace("SRR", "")).astype(int) >=794848) & (x.run_1_accession.apply(lambda row: row.replace("SRR", "")).astype(int) <= 794850)].to_csv("Escherichia_query_info.csv")

In [5]:
x.head(20).to_csv("Helicobacter_query_info.csv") #["experiment_title"]

In [18]:
x = EnaSearch(2, 20, "tuberculosis RNA")
x.search()
x = x.get_df()

In [19]:
x.head()

,study_accession,experiment_accession,experiment_title,description,tax_id,scientific_name,library_strategy,library_source,library_selection,sample_accession,sample_title,instrument_model,run_accession,read_count,base_count
0,PRJNA239223,SRX475349,Illumina HiSeq 2000 sequencing; GSM1333720: Ye...,Illumina HiSeq 2000 sequencing; GSM1333720: Ye...,502800,Yersinia pseudotuberculosis YPIII,RNA-Seq,TRANSCRIPTOMIC,cDNA,SAMN02650479,Yersinia pseudotuberculosis RNA-seq_early infe...,Illumina HiSeq 2000,SRR1175455,88333445,17843355890
1,PRJNA239223,SRX475349,Illumina HiSeq 2000 sequencing; GSM1333720: Ye...,Illumina HiSeq 2000 sequencing; GSM1333720: Ye...,502800,Yersinia pseudotuberculosis YPIII,RNA-Seq,TRANSCRIPTOMIC,cDNA,SAMN02650479,Yersinia pseudotuberculosis RNA-seq_early infe...,Illumina HiSeq 2000,SRR1175456,70676677,14276688754
2,PRJNA239223,SRX475350,Illumina HiSeq 2000 sequencing; GSM1333721: Y....,Illumina HiSeq 2000 sequencing; GSM1333721: Y....,502800,Yersinia pseudotuberculosis YPIII,RNA-Seq,TRANSCRIPTOMIC,cDNA,SAMN02650477,Y. pseudotuberculosis RNA-seq _persistent infe...,Illumina HiSeq 2000,SRR1175457,96794171,19552422542
3,PRJNA239223,SRX475350,Illumina HiSeq 2000 sequencing; GSM1333721: Y....,Illumina HiSeq 2000 sequencing; GSM1333721: Y....,502800,Yersinia pseudotuberculosis YPIII,RNA-Seq,TRANSCRIPTOMIC,cDNA,SAMN02650477,Y. pseudotuberculosis RNA-seq _persistent infe...,Illumina HiSeq 2000,SRR1175458,97439250,19682728500
4,PRJNA239223,SRX475351,Illumina HiSeq 2000 sequencing; GSM1333722: Y....,Illumina HiSeq 2000 sequencing; GSM1333722: Y....,502800,Yersinia pseudotuberculosis YPIII,RNA-Seq,TRANSCRIPTOMIC,cDNA,SAMN02650480,"Y. pseudotuberculosis RNA-seq_early infection,...",Illumina HiSeq 2000,SRR1175459,87517953,17678626506


### Sqlite approach

In [3]:
con = sqlite3.connect("SRAmetadb.sqlite")
cursor = con.cursor()
cursor.execute("select name from sqlite_master where type='table'")
print(cursor.fetchall())
# cur = con.cursor()

[('metaInfo',), ('submission',), ('study',), ('sample',), ('experiment',), ('run',), ('sra',), ('sra_ft',), ('sra_ft_content',), ('sra_ft_segments',), ('sra_ft_segdir',), ('col_desc',), ('fastq',), ('studies',), ('studies_data',), ('studies_idx',), ('studies_content',), ('studies_docsize',), ('studies_config',), ('sample_ft',), ('sample_ft_data',), ('sample_ft_idx',), ('sample_ft_content',), ('sample_ft_docsize',), ('sample_ft_config',), ('experiment_ft',), ('experiment_ft_data',), ('experiment_ft_idx',), ('experiment_ft_content',), ('experiment_ft_docsize',), ('experiment_ft_config',)]


In [58]:
# table = "sample"
# cols = "scientific_name,sample_accession"
# cursor.execute(f"create virtual table {table}_ft using fts5({cols})")
# cursor.execute(f"insert into {table}_ft select {cols} from {table}")

# table = "experiment"
# cols = "title,design_description,sample_accession,library_name,library_strategy"
# cursor.execute(f"create virtual table {table}_ft using fts5({cols})")
# cursor.execute(f"insert into {table}_ft select {cols} from {table}")

In [32]:
cols = "title,design_description,sample_accession,library_name,library_strategy"
exp_map = {i:ix for ix, i in enumerate(cols.split(","))}

### Query Process

In [49]:
# def search_genus_name(genus):

genus="Cronobacter"

query = f"""SELECT sample_accession 
        FROM sample_ft
        WHERE scientific_name
        MATCH '{genus}'
        """
samples = ",".join(["'" + i[0] + "'" for i in cursor.execute(query).fetchall()])

query = f"""SELECT *
        FROM experiment_ft
        WHERE sample_accession IN ({samples})
        AND experiment_ft MATCH 'RNA AND seq'
        """

# print(query)

results = cursor.execute(query).fetchall()

positive_terms = "wild WT".split()
negative_terms = "plasmid".split()

new_results = []
for i in results:
    descriptive_text = i[exp_map["title"]] + i[exp_map["library_name"]]
    add = True
    for n in negative_terms:
        if n in descriptive_text:
            add = False
            break
    if add:
        new_results.append(i)

results = new_results

samples = []

for ix, i in enumerate(results):
    descriptive_text = i[exp_map["title"]] + i[exp_map["library_name"]]
    for p in positive_terms:
        if p in descriptive_text:
            samples.append(i[exp_map["sample_accession"]])
print(samples)

def get_wigs_from_ftps(ftps, path="."):
    path = Path(path)
    for file in ftps:
        extracted = False
        wget.download(file)
        file = file.split("/")[-1]
        with tarfile.open(file, "r:") as tar:
            names = tar.getmembers()
            wig_files = [n for n in names if ".wig" in n.name]
            if len(wig_files) > 0:
                tar.extractall(path=path, members=wig_files)
                
        os.remove(file)
        

['SRS4102342']


In [ ]:
r = cursor.execute(f"select distinct library_layout from {table}")
rs = list(r.fetchall())
rs

In [17]:
rs[0][0]

'PAIRED - NOMINAL_SDEV: 24.7558; NOMINAL_LENGTH: 163; '

In [29]:
# [[i for i in r[0].split(";") if "NOMINAL" not in i]
max([len(list(r)) for r in rs])

1

In [6]:
table = "experiment"
field = "study_accession" #"sample_accession"
query = "SRP019956"

cs = [i[0] for i in cursor.execute(f"select * from {table}").description]
r = cursor.execute(f"select * from {table} where {field} = '{query}'")
rs = list(r.fetchone())
for ix , i in enumerate(cs):
    if True: # i in "library_strategy title design_description sample_accession".split():
        print(f"{i}: {rs[ix]}")

experiment_ID: 1225035.0
bamFile: None
fastqFTP: None
experiment_alias: GSM1104375_1
experiment_accession: SRX254733
broker_name: None
center_name: GEO
title: GSM1104375: Galaxy1-[CV104_pHDB3_5.txt]; Escherichia coli str. K-12 substr. MG1655; RNA-Seq
study_name: GSE45443
study_accession: SRP019956
design_description: None
sample_name: GSM1104375
sample_accession: SRS403764
sample_member: None
library_name: None
library_strategy: RNA-Seq
library_source: TRANSCRIPTOMIC
library_selection: cDNA
library_layout: SINGLE - 
targeted_loci: None
library_construction_protocol: None
spot_length: None
adapter_spec: None
read_spec: None
platform: ILLUMINA
instrument_model: Illumina HiSeq 2000
platform_parameters: INSTRUMENT_MODEL: Illumina HiSeq 2000
sequence_space: None
base_caller: None
quality_scorer: None
number_of_levels: None
multiplier: None
qtype: None
sra_link: None
experiment_url_link: GEO Sample: http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM1104375
xref_link: gds: 301104375
experi

In [ ]:
"""
may want to use the sample scientific name to boost the search of the experiment
"""

### Scratch Work

In [ ]:
for ix , i in enumerate(cs):
    if True: # i in "library_strategy title design_description sample_accession".split():
        print(f"{i}: {rs[ix]}\n")

In [11]:
def has_rna(row):
#     print(row)
    if row is None:
        return False
    if "RNA" in row or "rna" in row:
        return True
    else:
        return False

In [7]:
table.columns

Index(['sra_ID', 'SRR_bamFile', 'SRX_bamFile', 'SRX_fastqFTP', 'run_ID',
       'run_alias', 'run_accession', 'run_date', 'updated_date', 'spots',
       'bases', 'run_center', 'experiment_name', 'run_url_link',
       'run_entrez_link', 'run_attribute', 'experiment_ID', 'experiment_alias',
       'experiment_accession', 'experiment_title', 'study_name', 'sample_name',
       'design_description', 'library_name', 'library_strategy',
       'library_source', 'library_selection', 'library_layout',
       'library_construction_protocol', 'adapter_spec', 'read_spec',
       'platform', 'instrument_model', 'instrument_name',
       'platform_parameters', 'sequence_space', 'base_caller',
       'quality_scorer', 'number_of_levels', 'multiplier', 'qtype',
       'experiment_url_link', 'experiment_entrez_link', 'experiment_attribute',
       'sample_ID', 'sample_alias', 'sample_accession', 'taxon_id',
       'common_name', 'anonymized_name', 'individual_name', 'description',
       'sample_url

In [9]:
table["run_url_link run_entrez_link experiment_ID experiment_accession experiment_url_link experiment_entrez_link sample_ID sample_accession sample_url_link sample_entrez_link".split()].head(2)

,run_url_link,run_entrez_link,experiment_ID,experiment_accession,experiment_url_link,experiment_entrez_link,sample_ID,sample_accession,sample_url_link,sample_entrez_link
0,None,None,1.0,DRX000001,None,None,1.0,DRS000001,None,None
1,None,None,2.0,DRX000002,None,None,2.0,DRS000002,None,None


In [12]:
x = table[table["study_type"].apply(lambda row: has_rna(row))]

In [19]:
table.library_strategy.unique()

array(['WGS', 'FL-cDNA', 'CLONE', 'OTHER', 'POOLCLONE', 'ChIP-Seq', 'EST',
       'MNase-Seq', 'WCS', 'AMPLICON', 'RNA-Seq', 'CTS', 'WXS',
       'Bisulfite-Seq', 'MeDIP-Seq', 'miRNA-Seq', 'WGA', None,
       'FAIRE-seq', 'CLONEEND', 'ncRNA-Seq', 'RIP-Seq', 'RAD-Seq',
       'DNase-Hypersensitivity', 'SELEX', 'MBD-Seq', 'Targeted-Capture',
       'Tn-Seq', 'ATAC-seq', 'Synthetic-Long-Read',
       'Tethered Chromatin Conformation Capture', 'FINISHING', 'Hi-C',
       'ChIP', 'MRE-Seq', 'VALIDATION',
       'Targeted resequencing of human colorectal cancer (CRC) stem-like cells (CCSCs), stem-like circulating tumor cells (cCCSCs) and metastatic colorectal stem cells (mCCSCs).',
       'other', 'ChIA-PET'], dtype=object)

In [13]:
x["run_url_link run_entrez_link experiment_ID experiment_accession experiment_url_link experiment_entrez_link sample_ID sample_accession sample_url_link sample_entrez_link".split()].head(2)

,run_url_link,run_entrez_link,experiment_ID,experiment_accession,experiment_url_link,experiment_entrez_link,sample_ID,sample_accession,sample_url_link,sample_entrez_link
